# JS: Viewport Height

> JavaScript generator for dynamic viewport height calculation.

In [ ]:
#| default_exp js.viewport

In [ ]:
#| export
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds

## generate_viewport_height_js

Sets explicit pixel height on the card stack to fill available vertical space.
Measures sibling elements, parent padding, and space below to compute the
remaining height. Includes a debounced resize handler.

The `container_id` parameter is the consumer's parent container that holds
the card stack alongside header/footer elements. If not provided, the card
stack measures from its own position.

In [ ]:
#| export
def generate_viewport_height_js(
    ids: CardStackHtmlIds,  # HTML IDs for this card stack instance
    container_id: str = "",  # Consumer's parent container ID (empty = use card stack parent)
) -> str:  # JavaScript code fragment for viewport height calculation
    """Generate JS for dynamic viewport height calculation.
    
    Uses the browser's layout engine to measure the space consumed by sibling
    elements rather than summing individual heights. This naturally handles
    margin collapsing regardless of the container's display type.
    
    Strategy: temporarily collapse the card stack to 0 height, measure how
    much vertical space the remaining content occupies, then set the card
    stack height to fill the remaining viewport space.
    """
    # Build container resolution logic — guarded against null for SPA navigation
    if container_id:
        container_resolution = f"""
            const container = document.getElementById('{container_id}');"""
    else:
        container_resolution = f"""
            const _csEl = document.getElementById('{ids.card_stack}');
            if (!_csEl) return 400;
            const container = _csEl.parentElement;"""

    return f"""
        // === Viewport Height Calculation ===

        function calculateSpaceBelowContainer(container) {{
            let spaceUsedBelow = 0;
            let currentElement = container;
            let parent = container.parentElement;
            while (parent && parent !== document.documentElement) {{
                const parentStyles = getComputedStyle(parent);
                spaceUsedBelow += parseFloat(parentStyles.paddingBottom) || 0;
                spaceUsedBelow += parseFloat(parentStyles.borderBottomWidth) || 0;
                let foundCurrent = false;
                for (const sibling of parent.children) {{
                    if (sibling === currentElement) {{ foundCurrent = true; continue; }}
                    if (!foundCurrent) continue;
                    if (sibling.nodeType !== Node.ELEMENT_NODE) continue;
                    const tag = sibling.tagName;
                    if (tag === 'SCRIPT' || tag === 'STYLE') continue;
                    if (tag === 'INPUT' && sibling.type === 'hidden') continue;
                    const s = getComputedStyle(sibling);
                    if (s.display === 'none') continue;
                    const mt = parseFloat(s.marginTop) || 0;
                    const mb = parseFloat(s.marginBottom) || 0;
                    spaceUsedBelow += sibling.getBoundingClientRect().height + mt + mb;
                }}
                currentElement = parent;
                parent = parent.parentElement;
            }}
            return spaceUsedBelow;
        }}

        function calculateAndSetViewportHeight() {{
            {container_resolution}
            const cardStack = document.getElementById('{ids.card_stack}');
            const cardStackInner = document.getElementById('{ids.card_stack_inner}');
            if (!container || !cardStack) return 400;

            // Temporarily collapse the card stack to measure sibling space.
            // Save current values to restore if something goes wrong.
            const prevHeight = cardStack.style.height;
            const prevMinHeight = cardStack.style.minHeight;
            const prevMaxHeight = cardStack.style.maxHeight;
            
            cardStack.style.height = '0px';
            cardStack.style.minHeight = '0px';
            cardStack.style.maxHeight = '0px';

            // Now measure: with the card stack collapsed, the container's
            // height is driven entirely by the sibling content + padding.
            // The browser handles margin collapsing for us.
            const containerRect = container.getBoundingClientRect();
            const siblingSpace = containerRect.height;

            // Restore immediately (before any repaint)
            cardStack.style.height = prevHeight;
            cardStack.style.minHeight = prevMinHeight;
            cardStack.style.maxHeight = prevMaxHeight;

            const windowHeight = window.innerHeight;
            const containerTop = containerRect.top;
            const spaceBelow = calculateSpaceBelowContainer(container);

            const viewportHeight = Math.floor(Math.max(200,
                windowHeight - containerTop - siblingSpace - spaceBelow
            ));

            cardStack.style.height = viewportHeight + 'px';
            cardStack.style.maxHeight = viewportHeight + 'px';
            cardStack.style.minHeight = viewportHeight + 'px';
            if (cardStackInner) cardStackInner.style.height = viewportHeight + 'px';

            return viewportHeight;
        }}

        function _debounce(fn, delay) {{
            let tid;
            return function(...args) {{ clearTimeout(tid); tid = setTimeout(() => fn.apply(this, args), delay); }};
        }}

        const _debouncedResize = _debounce(calculateAndSetViewportHeight, 100);

        if (!window._csResizeListener_{ids.prefix.replace('-', '_')}) {{
            window._csResizeListener_{ids.prefix.replace('-', '_')} = true;
            window.addEventListener('resize', _debouncedResize);
        }}

        // Expose on namespace
        ns.recalculateHeight = calculateAndSetViewportHeight;
    """

In [ ]:
# Test that the generator produces valid JS referencing correct IDs
ids = CardStackHtmlIds(prefix="cs0")
js = generate_viewport_height_js(ids, container_id="my-container")
assert "my-container" in js
assert ids.card_stack in js
assert ids.card_stack_inner in js
assert "ns.recalculateHeight" in js
assert "calculateAndSetViewportHeight" in js
# Verify collapse-and-measure strategy
assert "0px" in js  # Temporarily collapses card stack
assert "siblingSpace" in js
assert "spaceBelow" in js
print("Viewport height JS generator tests passed!")

Viewport height JS generator tests passed!


In [ ]:
# Test without container_id (falls back to parent element with null guard)
js_no_container = generate_viewport_height_js(ids)
assert ".parentElement" in js_no_container
# Verify null guard exists before .parentElement access
assert "if (!_csEl) return 400" in js_no_container
# Verify collapse-and-measure is present in both paths
assert "siblingSpace" in js_no_container
print("Viewport height JS fallback container test passed!")

Viewport height JS fallback container test passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()